# 데이터셋 만들기 (기업 재무정보)

In [ ]:
import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime

# crtfc_key=

#회사정보 가져오기
#임의로 'E:/' 로 설정 및 다운로드 파일 corpcode.zip으로 설정
path='data'
filename='corpcode.zip'

url = 'https://opendart.fss.or.kr/api/corpCode.xml'
params = {
    'crtfc_key': crtfc_key,
}
results=requests.get(url, params=params)

file = open(path+filename, 'wb')
file.write(results.content)
file.close()

## 내 폴더에 저장하기 


In [ ]:
path='data'

filename='corpcode.zip'

In [ ]:
import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime
zipfile.ZipFile(path+filename).extractall(path)


tree = parse(path + '/CORPCODE.xml') #상장회사 이름과 티커 들고오기 
root = tree.getroot()
li=root.findall('list')
corp_code,corp_name,stock_code,modify_date=[],[],[],[]
for d in li:
    corp_code.append(d.find('corp_code').text)
    corp_name.append(d.find('corp_name').text)
    stock_code.append(d.find('stock_code').text)
    modify_date.append(d.find('modify_date').text)
corps_df = pd.DataFrame({'corp_code':corp_code,'corp_name':corp_name,
         'stock_code':stock_code,'modify_date':modify_date})

corps_df = corps_df.loc[corps_df['stock_code']!=' ',:].reset_index(drop=True)

In [4]:
corps_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00359614,리더컴,056140,20170630


## 추출한 회사 이름으로 Dart에서 재무정보 들고오기
다양한 재무정보가 존재하지만 Text로 쓰기 용이한 데이터는 "회사의 개요, 사업의 내용, 경영진단 및 분석" 이다 
이 데이터는 api로 제공되지 않음으로 직접 크롤링 진행

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
request_list = ['A001','A002','A003'] # 각각 회사의 개요, 사업의 내용, 경영진단 및 분석 페이지 형식
final_list = []
error_list = []
# 메인 페이지 URL
for i,r in tqdm(corps_df[['corp_code','corp_name']].iterrows()):
    text_list = [r.iloc[1]]
    for i in request_list:
        url = f"https://dart.fss.or.kr/navi/searchNavi.do?naviCrpCik={r.iloc[0]}&naviCrpNm={r.iloc[1]}&naviCode={i}" 
        headers = {
        'User-Agent': 'Mozilla/5.0',
        'Connection': 'keep-alive'  # keep-alive 설정
        }
        # 메인 페이지 요청
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # iframe URL 추출
        iframe_tag = soup.find('iframe')
        iframe_url = iframe_tag['src'] if iframe_tag else None

        if iframe_url:
            
            # 절대 경로로 변환
            iframe_url = f"https://dart.fss.or.kr{iframe_url}" if iframe_url.startswith('/') else iframe_url
            
            # iframe 페이지 요청
            iframe_response = requests.get(iframe_url, headers=headers)
            
            # iframe 소스를 파싱
            iframe_soup = BeautifulSoup(iframe_response.text, 'html.parser')
            iframe_text = iframe_soup.get_text(separator='\n', strip=True)
            text_list.append(iframe_text)
        else:
            text_list = []
            error_list.append(r.iloc[1])
        time.sleep(0.5)
    if text_list == []:
        pass
    else:
        final_list.append(text_list)

3784it [2:21:53,  2.25s/it]


In [ ]:
final_list

## 데이터 전처리

In [ ]:
import pandas as pd
final_df = pd.read_csv('/Users/sonhyeongjin/Desktop/project/KRX_LLM/finance_company_report.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sonhyeongjin/Desktop/project/KRX_LLM/finance_company_report.csv'

In [ ]:
final_df

In [ ]:
final_df = final_df.set_index('0')

KeyError: "None of ['0'] are in the columns"

### 회사의 개요
회사의 개요 같은 경우 비어있는 형식도 다양하고 정보량 수준이 다 달랐음 일괄적으로 처리하기 어려워 문장을 임베딩 모델에 넣어 확인함

In [258]:
print(final_df.iloc[1,:][0]) # 내용이 없음

1. 회사의 개요
1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이 사업보고서, 분기ㆍ반기보고서를 제출하는 경우에 한함)
연결대상 종속회사 현황(요약)
(단위 : 사)
구분
연결대상회사수
주요
종속회사수
기초
증가
감소
기말
상장
-
-
-
-
-
비상장
-
-
-
-
-
합계
-
-
-
-
-
※상세 현황은 '상세표-1. 연결대상 종속회사 현황(상세)' 참조
1-1. 연결대상회사의 변동내용
구 분
자회사
사 유
신규
연결
-
-
-
-
연결
제외
-
-
-
-
중소기업 등 해당 여부
중소기업 해당 여부
미해당
벤처기업 해당 여부
미해당
중견기업 해당 여부
미해당
회사의 주권상장(또는 등록ㆍ지정)여부 및 특례상장에 관한 사항
주권상장
(또는 등록ㆍ지정)여부
주권상장
(또는 등록ㆍ지정)일자
특례상장 등
여부
특례상장 등
적용법규
-
-
-
-


C:\Users\son\AppData\Local\Temp\ipykernel_10340\2338289525.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(final_df.iloc[1,:][0]) # 내용이 없음


In [16]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import collections

from tqdm import tqdm

# KoBigBird 모델 로드
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base")



# 텍스트 데이터를 임베딩으로 변환하는 함수
def get_embeddings(texts):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)
        with torch.no_grad():
            model_output = model(**inputs)
            # CLS 토큰의 출력을 사용해 임베딩 생성
            embedding = model_output.last_hidden_state[:, 0, :].squeeze().numpy()
            embeddings.append(embedding)
    return np.array(embeddings)

# 예시 텍스트 데이터 배열
texts = final_df.loc[:,'1'].values
texts = np.array([text.replace("\n", " ") for text in texts])
# 텍스트를 임베딩으로 변환
embeddings = get_embeddings(texts)



100%|██████████| 2984/2984 [23:58<00:00,  2.07it/s]  


In [17]:
num_clusters = 5  # 군집 수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(embeddings)

# 각 데이터가 속한 군집 번호
cluster_labels = kmeans.labels_

# 2. KNN으로 유사도 평가
knn = KNeighborsClassifier(n_neighbors=3)  # 이웃 수 조정 가능
knn.fit(embeddings, cluster_labels)

# 각 군집에 데이터가 몇 개씩 들어 있는지 계산
cluster_counts = collections.Counter(cluster_labels)

# 결과 출력
print("Cluster counts:", cluster_counts)
print("Cluster labels for each data point:", cluster_labels)

# 각 데이터가 속한 군집 및 군집별 개수 확인
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count} points")

# 예시: 새로운 데이터 포인트와 가까운 군집 찾기


/Users/sonhyeongjin/miniforge3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cluster counts: Counter({2: 1578, 1: 555, 4: 374, 3: 243, 0: 234})
Cluster labels for each data point: [3 3 2 ... 4 2 0]
Cluster 3: 243 points
Cluster 2: 1578 points
Cluster 4: 374 points
Cluster 0: 234 points
Cluster 1: 555 points


In [18]:
count = 0
cluster_data = []
for i in cluster_labels:
    if i == 3:
        cluster_data.append(texts[count])
    count += 1

In [ ]:
cluster_data

클러스터 수 5개정도면 유의미할 것 같음 
초기에 확인한 의미없는 데이터들이 3번 군집에 많이 있음을 확인 => 3번 클러스터를 np.NaN값으로 대체  

In [20]:
count = 0
cluster_data = []
item1_replace = []
for i in cluster_labels:
    if i == 3:
        item1_replace.append(np.NaN)
    else:
        item1_replace.append(texts[count])
    count += 1

In [ ]:
item1_replace

In [22]:
final_df['item1_replace'] = item1_replace

In [24]:
# final_df.to_csv('finance_company_report_replace1.csv')

In [283]:
import pandas as pd
final_df = pd.read_csv('finance_company_report_replace1.csv')

In [289]:
final_df[final_df['0'] == '드래곤플라이']['item1_replace'].values

array(["1. 회사의 개요 (1) 연결대상 종속회사 현황 1) 연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결대상회사수 주요 종속회사수 기초 증가 감소 기말 상장 - - - - - 비상장 3 - - 3 1 합계 3 - - 3 1 ※상세 현황은 '상세표-1. 연결대상 종속회사 현황(상세)' 참조 2) 연결대상회사의 변동내용 구 분 자회사 사 유 신규 연결 - - - - 연결 제외 - - - - (2) 회사의 법적ㆍ상업적 명칭 당사의 명칭은「주식회사 드래곤플라이」라고 표기합니다. 또한, 영문으로는「Dragonfly GF Co., Ltd.(약호 Dragonfly)」라고 표기합니다. (3) 설립일자 당사는 1990년 3월 22일에 설립되었습니다. 또한 1997년 11월 10일자로 코스닥시장에 상장되었으며, 2009년 7월 10일자로 (구)드래곤플라이를 흡수합병하였습니다. (4) 본사의 주소, 전화번호, 홈페이지 주소 1) 주소 : 서울특별시 서초구 서초중앙로22길 80(서초동) 2) 전화번호 : 02) 522-9393 3) 홈페이지 : http://www.dragonflygame.com (5) 중소기업 등 해당 여부 중소기업 해당 여부 해당 벤처기업 해당 여부 미해당 중견기업 해당 여부 미해당 (6) 주요 사업의 내용 당사는「VR(가상현실), AR(증강현실), 온라인, 모바일 콘텐츠 소프트웨어의 개발 및유통업」을 주요 사업목적으로 영위 중이며, 멀티 플랫폼 기반의「VRㆍAR 게임」및 PC 인터넷 플랫폼 기반의「온라인 게임」과 스마트폰 등 모바일 플랫폼 기반의「모바일 게임」을 개발 및 유통하고 있습니다. 현재, 유통 또는 개발 중인 주요 게임 콘텐츠로는「스페셜포스 VR: INVASION」,「스페셜포스 VR: INFINITY WAR」등의 VRㆍAR 게임 및「스페셜포스」,「스페셜포스2」,「프로젝트 AX」등의 온라인 게임과「스페셜포스 서바이벌M」,「콜 오브 카오스 : 어셈블」,「아도르: 수호의 여신」,「위치스위치」등의 모바일 게임이 있습니다. 당사의 종속회사인

## 사업의 내용
사업의 내용같은 경우는 대부분 데이터가 차 있는 것 같아서 아에 정보가 존재하지 않는 데이터만 제외하는 형식으로 진행한다.

In [290]:
import numpy as np
texts = final_df.loc[:,'2'].values
texts = np.array([text.replace("\n", " ") for text in texts])

In [291]:
texts[1]

'II. 사업의 내용 1. 사업의 개요 2. 주요 제품 및 서비스 3. 원재료 및 생산설비 4. 매출 및 수주상황 5. 위험관리 및 파생거래 6. 주요계약 및 연구개발활동 7. 기타 참고사항'

In [292]:
texts_normal = np.array([text.replace(texts[1], 'a') for text in texts])

In [293]:
item2_list = []
count =0
count2 = 0
for i in texts_normal:
    if i == 'a':
        item2_list.append(np.NaN)
        count2 += 1
    else:
        item2_list.append(i)

In [294]:
item2_list[:10]

['II. 사업의 내용 1. 사업의 개요 가. 영업의 현황 엠씨스퀘어 사업이 35 년의 오랜 시간을 겪어오며 여러 어려운 과정이 있었지만 새로운 성장 분야를 위해 끊임없는 노력을 계속하고 있으며 , 뇌과학 기술에 대한 연구와 임상 시험에 대한 투자를 소홀히 하지 않고 그동안 많은 준비해 왔습니다 . 엠씨스퀘어는 뇌과학기술을 기초로한 헬스케어 분야에 오랜 경험과 사업의 뿌리를 갖고 있습니다 . 성장분야로서 수면시장이 크게 확장되고 있고 또한 반려견 시장에서 반려견들의 정서안정 등 응용 분야에 대한 기회가 열리고 있습니다 . 수면시장과 반려견들에 대해 그동안 구축해온 임상 Evidence 를 주축으로 B2C 및 B2B 사업 분야로 신규 사업 기회가 확장되가며 디자인 전문회사 및 유통 전문 회사들과 전략적 제휴를 통해 수면 및 반려견 관련 새로운 제품 라인업을 구성하며 사업을 진행해 가고 있습니다 . 한편으로 회사의 재무구조를 개선하기 위하여 , 페인스크램블러 의료기기 사업 부분을 관련 바이오전문기업에게 자산양수도 방식으로 채무 상환에 합의하여 구조 조정하여 사업 양도 시켰으며 , 이에 따라 회사의 핵심 역량을 엠씨스퀘어에 집중하여 성장해 가고 있는 디지털 헬스 시장에 회사의 모든 역량을 집중하여 수면시장과 반려견 시장을 중점으로 진행해 가고 있습니다 . 엠씨스퀘어 사업 주요 진행 현황 B2C 사업으로 홈쇼핑 전문유통회사와의 계약을 체결하고 상품 디자인을 위한 전문 디자인회사와 전략적 제휴를 통하여 수면 전문 제품과 펫 정서안정을 위한 제품을 개발하여 사업 진행 중에 있습니다 . 당사는 엠씨스퀘어 기술 제공으로 로얄티를 받는 계약 구조로 상품 생산에 따른 자금 부담을 최소화하고 선 주문에 의한 개발 진행으로 사업리스크가 최소화된 사업 형태로 진행 중에 있습니다 . 유통회사의 선 투자에 의한 제품 개발 형태로서 새로운 사업 구조로 진행하고 있으며 , 당사의 기술제공 및 유통회사의 생산 및 마케팅 투자로 상호 강점을 결합하여 사업을 확장해 가는 새로운 사업구조입니다 .

In [295]:
final_df['item2_replace'] = item2_list

In [296]:
final_df

,0,1,2,3,item1_replace,item2_replace
0,지오엠씨,1. 회사의 개요\n1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인...,II. 사업의 내용\n1. 사업의 개요\n가. 영업의 현황\n엠씨스퀘어 사업이\n3...,IV. 이사의 경영진단 및 분석의견\n- 기업공시서식 작성기준에 따라 분/반기 보고...,NaN,II. 사업의 내용 1. 사업의 개요 가. 영업의 현황 엠씨스퀘어 사업이 35 년의...
1,클루넷,1. 회사의 개요\n1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인...,II. 사업의 내용\n1. 사업의 개요\n2. 주요 제품 및 서비스\n3. 원재료 ...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n당사가 동 사...,NaN,NaN
2,GRT,1. 회사의 개요\n가. 연결대상 종속회사 개황\n(연결재무제표를 작성하는 주권상장...,"II. 사업의 내용\n1. 사업의 개요\n당사는 홍콩에 설립된 비금융 지주회사이며,...",IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n당사가 이사의...,1. 회사의 개요 가. 연결대상 종속회사 개황 (연결재무제표를 작성하는 주권상장법인...,"II. 사업의 내용 1. 사업의 개요 당사는 홍콩에 설립된 비금융 지주회사이며, 실..."
3,보진재,1. 회사의 개요\n(1) 회사의 법적 및 상업적 명칭\n당사의 국문 명칭은 주식회...,II. 사업의 내용\n1. 사업의 개요\n(\n1) 사업부문별 시장여건 및 영업의 ...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n당사가 동\n...,1. 회사의 개요 (1) 회사의 법적 및 상업적 명칭 당사의 국문 명칭은 주식회사 ...,II. 사업의 내용 1. 사업의 개요 ( 1) 사업부문별 시장여건 및 영업의 개황 ...
4,로스웰,1. 회사의 개요\n1. 연결대상 종속회사 개황\n(1) 연결대상 종속회사 현황(요...,II. 사업의 내용\n1. 사업의 개요\n당사와 연결종속회사의 주된 영업활동은 자동...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n본 자료는 미...,1. 회사의 개요 1. 연결대상 종속회사 개황 (1) 연결대상 종속회사 현황(요약)...,II. 사업의 내용 1. 사업의 개요 당사와 연결종속회사의 주된 영업활동은 자동차용...
...,...,...,...,...,...,...
2979,HJ중공업,1. 회사의 개요\n가. 연결대상 종속회사 현황(요약)\n(단위 : 사)\n구분\n...,II. 사업의 내용\n1. 사업의 개요\n당사는 조선업과 건설업을 주요사업으로 영위...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n당사가 동 보...,1. 회사의 개요 가. 연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결대상...,II. 사업의 내용 1. 사업의 개요 당사는 조선업과 건설업을 주요사업으로 영위하고...
2980,라이온켐텍,1. 회사의 개요\n1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인...,II. 사업의 내용\n1. 사업의 개요\n당사는 건축 내외장재로 사용되는 인조대리석...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의 사항\n동 사업보고...,1. 회사의 개요 1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이...,II. 사업의 내용 1. 사업의 개요 당사는 건축 내외장재로 사용되는 인조대리석 및...
2981,에어부산,1. 회사의 개요\n가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인...,II. 사업의 내용\n1. 사업의 개요\n당사의 주요 제품 및 서비스는 여객운송서비...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 유의사항\n본 자료는 미...,1. 회사의 개요 가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이...,II. 사업의 내용 1. 사업의 개요 당사의 주요 제품 및 서비스는 여객운송서비스입...
2982,화인베스틸,1. 회사의 개요\n가. 연결대상 종속회사 개황\n1) 연결대상 종속회사 현황(요약...,II. 사업의 내용\n1. 사업의 개요\n당사는 단일 사업부문(형강 제조 및 판매)...,IV. 이사의 경영진단 및 분석의견\n1. 예측정보에 대한 주의사항\n당사가 동 이...,1. 회사의 개요 가. 연결대상 종속회사 개황 1) 연결대상 종속회사 현황(요약) ...,II. 사업의 내용 1. 사업의 개요 당사는 단일 사업부문(형강 제조 및 판매)으로...


### 경영진단
분/반기 보고서 경우에 기제하지 않음

In [297]:
import numpy as np
texts = final_df.loc[:,'3'].values
texts = np.array([text.replace("\n", " ") for text in texts])

In [298]:
import re
def process_sentence(sentence):
    # "2."이 있는지 확인
    if "2." in sentence:
        # "2."을 기준으로 분할하여 뒷부분을 가져옴
        return re.split(r"2\.", sentence, maxsplit=1)[-1].strip()
    else:
        # "2."이 없는 경우 np.NaN 반환
        return np.NaN

processed_sentences = np.array([process_sentence(sentence) for sentence in texts], dtype=object)



In [299]:
processed_sentences

array([nan,
       "개요 당사는 인터넷 인프라 서비스 전문 기업으로 클라우드 컴퓨팅 기술인 CCN(Cloud Computing Network)기반의 CDN(Contents Delivery Network)서비스를 중점적으로 개발, 공급하고 있으며 클라우드 스토리지 서비스와 디지털 콘텐츠 유통을 주요 사업으로 영위하고 있습니다. 기업부설연구소 체제로 지속적인 기술 개발 및 신제품 출시를 위해 노력하고 있습니다. 실제 CDN 기반의 콘텐츠 전송기술에 클라우드 컴퓨팅 기술을 접목시켜 CCN 이라는 서비스로 초기 시장을 선점 하고 있으며 클라우드 스토리지 서비스 등 다양한 비지니스 모델을 만들어 가고 있습니다. 당사의 주요고객은 온라인 콘텐츠 유통 업체를 비롯해 온라인 교육 사업자와 게임업체 및 포털업체 그리고 인터넷방송사에 이르기까지 대용량의 콘텐츠 사업을 하고 있는 곳입니다. 기타 자세한 사항은 동 신고서의 'Ⅱ. 사업의 내용'을 참조하시기 바랍니다. 3. 재무상태 및 영업실적에 대한 분석 당사의 당기 (제19기 : 2020. 01. 01 - 2020. 12. 31) 경영성과를 전기 (제18기 : 2019.01.01 - 2019. 12. 31) 와 대비하여 진단하였습니다. (1) 경영성과 2020년 매출액은 0백만원으로 전년도 0백만원 대비 약0% 감소 하였으며 영업손실은 전년동기 21만원에서 19백만원으로 손실폭이 축소되었습니다. 지분법손실로 인해 당기순손실 역시 전년동기 186백만원에서 당기 152백만원으로 전년대비\xa0 34백만원으로 손실이 축소 되었습니다. - 당사의 영업실적에 관한 자료는 다음과 같습니다. (단위 : 원) 구\xa0 \xa0\xa0 분 영\xa0 \xa0 \xa0 업\xa0 \xa0 \xa0 실\xa0 \xa0 \xa0 적 증감금액 증감비율 (%) 2020년 2019년 매출액 - - - - 매출원가 - - - - 매출총이익 - - - - 판매관리비 17,681,320 18,517,203 -835,883 -9.62% 영업이익

In [300]:
final_df['item3_replace'] = processed_sentences

In [301]:
final_df.isna().sum()

0                  0
1                  0
2                  0
3                  0
item1_replace    243
item2_replace      1
item3_replace     59
dtype: int64

In [309]:
final_df[final_df['0'] == '드래곤플라이']['item2_replace'].values

array(["II. 사업의 내용 1. 사업의 개요 [게임 사업] 당사와 당사의 종속회사인 (주)디에프랩은 「VR(가상현실), AR(증강현실), 온라인, 모바일 콘텐츠 소프트웨어의 개발 및유통업」을 주요 사업목적으로 영위 중이며, 멀티 플랫폼 기반의「VRㆍAR 게임」및 PC 인터넷 플랫폼 기반의「온라인 게임」과 스마트폰 등 모바일 플랫폼 기반의「모바일 게임」을 개발 및 유통하고 있습니다. 당사의 주요 매출 품목인 스페셜포스, 콜오브카오스 등의 온라인, 모바일 게임 등의 매출은 당반기 연결기준 15,068백만원(내수 14,618백만원, 수출 450백만원)을 시현하며 전체매출의 90.29%를 차지하였습니다. 온라인 및 모바일 게임은 직접 서비스 하는 경우와 외부에서 퍼블리싱 하는 경우, 외부 게임을 당사에서 퍼블리싱 하는 경우로 구분할 수 있는데, 당사의 '스페셜포스'의 경우 국내서비스를 자체적으로 운영하고 있으며, '스페셜포스2'는 국내 게임포탈인 넷마블에서 서비스 하고 있습니다. 해외에서는 해외 각국의 퍼블리셔에서 서비스 중입니다. '콜오브카오스 어셈블'의 경우 당사에서 퍼블리싱을 하고 있습니다. VRㆍAR 콘텐츠의 경우 스팀 등 온라인 플랫폼을 통하여 개인 직접 판매, 해외수출 등의 방법으로 판매하고 있습니다. [기능성 필름 및 자동차 흡음재 사업] 당사의 종속회사인 (주)리노펙은 기능성 필름 사업과 자동차 흡음재 사업을 목적으로 영위중이며, Roll-to-Roll 스퍼터 시스템을 이용한 ITO(Indium Tin Oxide)를 증착시킨 '투명전도막 필름'을 개발하여 현대엘리베이터 필름, LG전자 세탁기 증착필름, LG하우시스 고내식성 세탁기 적용필름 위주로 생산하고 있습니다. 또한, 30년간 축적된 자동차 패드류 생산, 제조 기술력을 활용하여 흡음재, 쿠션 및 씰링 제품군을 생산하고 있습니다. 당반기 연결기준 동사업부 기능성필름과 자동차 흡음재는 각각 606백만원, 1,015백만원의 매출을 시현하였으며, 모두 국내 매출로 각각 전체매출의 3.63%와 6.0

## 학습용 데이터 만들기 

In [ ]:
import os
# os.environ['OPENAI_API_KEY'] = 

In [15]:
final_df_slice = final_df[['0','item1_replace','item2_replace','item3_replace']]

In [16]:
final_df_slice

,0,item1_replace,item2_replace,item3_replace
0,지오엠씨,NaN,II. 사업의 내용 1. 사업의 개요 가. 영업의 현황 엠씨스퀘어 사업이 35 년의...,NaN
1,클루넷,NaN,NaN,개요 당사는 인터넷 인프라 서비스 전문 기업으로 클라우드 컴퓨팅 기술인 CCN(Cl...
2,GRT,1. 회사의 개요 가. 연결대상 종속회사 개황 (연결재무제표를 작성하는 주권상장법인...,"II. 사업의 내용 1. 사업의 개요 당사는 홍콩에 설립된 비금융 지주회사이며, 실...","개요 당사의 제 1공장, 2공장, 3공장은 모두 가동중입니다. 당사의 당기 매출액은..."
3,보진재,1. 회사의 개요 (1) 회사의 법적 및 상업적 명칭 당사의 국문 명칭은 주식회사 ...,II. 사업의 내용 1. 사업의 개요 ( 1) 사업부문별 시장여건 및 영업의 개황 ...,개 요 회사의 경영에 대한 전반적인 사항을 파악하기 위하여 장부와 관계서류를 열람하...
4,로스웰,1. 회사의 개요 1. 연결대상 종속회사 개황 (1) 연결대상 종속회사 현황(요약)...,II. 사업의 내용 1. 사업의 개요 당사와 연결종속회사의 주된 영업활동은 자동차용...,개요 당사와 연결종속회사의 주된 영업활동은 자동차용 전장제품의 연구개발과 제조 및 ...
...,...,...,...,...
2979,HJ중공업,1. 회사의 개요 가. 연결대상 종속회사 현황(요약) (단위 : 사) 구분 연결대상...,II. 사업의 내용 1. 사업의 개요 당사는 조선업과 건설업을 주요사업으로 영위하고...,"개요 당사는 조선업(선박건조)과 건설업(토목, 건축, 주택, 플랜트 등)을 주요 사..."
2980,라이온켐텍,1. 회사의 개요 1. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이...,II. 사업의 내용 1. 사업의 개요 당사는 건축 내외장재로 사용되는 인조대리석 및...,개요 당사는 1973년 새한화학공업사로 설립 이후 연구개발을 지속적으로 수행하여 1...
2981,에어부산,1. 회사의 개요 가. 연결대상 종속회사 개황(연결재무제표를 작성하는 주권상장법인이...,II. 사업의 내용 1. 사업의 개요 당사의 주요 제품 및 서비스는 여객운송서비스입...,"개요 당사는 2019년 연간 매출액은 전년대비 3.1% 감소한 6,332억 원, 영..."
2982,화인베스틸,1. 회사의 개요 가. 연결대상 종속회사 개황 1) 연결대상 종속회사 현황(요약) ...,II. 사업의 내용 1. 사업의 개요 당사는 단일 사업부문(형강 제조 및 판매)으로...,개요 회사의 경영에 대한 전반적인 사항을 파악하기 위하여 장부와 관계서류를 열람하고...


### open ai 입력 데이터 만들기

In [25]:
import math
qrys = []
for i in range(len(final_df_slice.index)):
    for j in range(1,len(final_df_slice.columns)):
        check_company = final_df_slice.iloc[i,0]
        check_text = final_df_slice.iloc[i,j]
        if type(check_text) == np.str_ or type(check_text) == str:
          
            messages = [
            {"content": f"당신의 임무는  정량적 금융 문제를 만드는 것입니다. 기업의 공시 정보를 제공하며 한국어로 시험형 문제로 재구성해보세요. 질문은 앞에 Q.를 붙이고 답변은 A.를 붙이세요 여러개의 질문 답변쌍을 내용이 겹치지 않게 만드세요. 제공된 공시 기업 이름은 {check_company}입니다.", "role": "system"},
            {"content": check_text, "role": "user"}]
            qrys.append(messages)

            


In [26]:
len(qrys)

8649

In [43]:
import pickle
with open('qrys.pkl',"wb") as file:
    pickle.dump(qrys,file)

<!--  -->

In [19]:
# import pickle
# with open('qrys.pkl','rb') as file:
#     qrys = pickle.load(file)

In [27]:
import random

# 리스트 qrys의 원소들을 랜덤하게 섞음
random.shuffle(qrys)


In [ ]:


import time
import openai
from litellm import completion, batch_completion
import litellm
from tqdm import tqdm 
# messages 생성 및 토큰 계산
responses = []
batch_size = 2  # 한번에 요청할 batch 크기
sleep_interval = 1  # 각 배치 사이의 슬립 시간 (초)
count = 0
# litellm의 verbose 설정
litellm.set_verbose = False  # 초기 설정은 False로

for i in tqdm(range(0, len(qrys), batch_size)):
    batch_messages = qrys[i:i + batch_size]

    batch_responses = batch_completion(
        model="gpt-4o-mini-2024-07-18",
        messages=batch_messages
    )
    if 'RateLimitError' in str(batch_responses):  # 허용 호출수준이 넘어가면 자동으로 끊기는게 아니라 그냥 오류를 출력함 => 비용, 시간 낭비 임으로 예외 처리 
        break
    else :
        responses.extend(batch_responses)
        count += 1

    # "Give Feedback / Get Help" 메시지 확인


    time.sleep(sleep_interval)

  0%|          | 11/4325 [02:26<14:35:08, 12.17s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



 66%|██████▋   | 2876/4325 [9:31:29<3:34:31,  8.88s/it] 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



 67%|██████▋   | 2917/4325 [9:38:50<3:59:57, 10.23s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



 82%|████████▏ | 3568/4325 [11:29:19<2:00:16,  9.53s/it]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



 82%|████████▏ | 3568/4325 [11:29:24<2:26:16, 11.59s/it]


In [ ]:
# import pickle

# with open('data/gpt_4o_korcompany_qa2024.pkl', 'wb') as file:
#     pickle.dump(responses,file)


In [24]:
import pickle

# pickle 파일 열기
with open("data/gpt_4o_korcompany_qa2024.pkl", "rb") as file:  # "rb"는 읽기와 바이너리 모드를 의미
    data = pickle.load(file)

TypeError: ContextWindowExceededError.__init__() missing 2 required positional arguments: 'model' and 'llm_provider'

In [25]:
import pickle

# Safe Unpickling을 위한 사용자 정의 클래스 무시
class ContextWindowExceededError(Exception):
    def __init__(self, *args, **kwargs):
        pass

# pickle 파일 열기
with open("data/gpt_4o_korcompany_qa2024.pkl", "rb") as file:
    try:
        data = pickle.load(file)
    except Exception as e:
        print(f"Error while loading pickle: {e}")

Error while loading pickle: ContextWindowExceededError.__init__() missing 2 required positional arguments: 'model' and 'llm_provider'


In [19]:
import pandas as pd

In [89]:
df_cfa = pd.read_csv('data/math/CFA_KRX.csv')

In [98]:
df_cfa_clean = df_cfa.iloc[:,1:]

In [99]:
df_cfa_clean.columns = ['question','response']

In [100]:
df_cfa_clean

,question,response
0,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:재무 제표 분...,C
1,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:비율은 재무 ...,C
2,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:분석가가 사용...,A
3,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:회사가 발표 ...,C
4,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:독립적 인 감...,B
...,...,...
1027,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:고객은 지금부...,B
1028,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:오늘 투자 한...,B
1029,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:은행은 명시된...,B
1030,다음 문제를 읽고 정답으로 가장 알맞은 것을 고르시요.\n### 질문:다음 위험 보...,C


In [192]:

extra_data_krx_sample_1_make_gpt_indexing = df_cfa_clean[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

In [193]:
extra_data_krx_sample_1_make_gpt__Dataset.save_to_disk("data/final/cfa_woobin")

Saving the dataset (0/1 shards):   0%|          | 0/1032 [00:00<?, ? examples/s]

In [104]:
df_math = pd.read_csv('data/math/kmwp train.csv')

In [113]:
df_math_del8 = df_math[~(df_math['class'] == 8)]

In [127]:
df_math_del8_clean = pd.DataFrame([df_math_del8['problem'],'파이썬 코드는 아래와 같고 '+df_math_del8['code'] +" 정답은 다음과 같음 "+ df_math_del8['answer']]).T

In [128]:
df_math_del8_clean.columns = ['question','response']

In [123]:
df_math_del8['problem']

1       윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...
3       6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...
4                                    0.26 x 0.8을 계산해 주세요.
5       현수네 집에서 미용실, 병원, 백화점까지의 거리는 각각 4/5km, 1/3km, 0...
6       어떤 수를 3으로 나누어야 하는 것을 잘못하여 9로 나누었더니 몫이 9가 되었습니다...
                              ...                        
2813    어느 가전 마트에서는 115만 원인 노트북을 어제까지는 30%를 할인하여 판매하였습...
2815    10분 동안 6.53k미터를 가는 자동차와 20분 동안 15.12k미터를 가는 오토...
2817           4880보다 ○ 큰 수 는 9000입니다. ○안에 알맞은 수를 써 넣으시오.
2818    어느 문구점에서 연필 한 자루는 400원이고 공책 한 권은 1000원입니다. 현승이...
2819    초콜릿 44개를 상자 11개에 똑같이 나누어 담으려고 합니다. 한 상자에는 몇 개에...
Name: problem, Length: 2020, dtype: object

In [144]:
df_math_del8_clean.reset_index(inplace=True)

In [194]:
df_math_del8_clean.iloc[:,2:]

,question,response
0,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,파이썬 코드는 아래와 같고 a = 8\nb = 16\ny = b // a\nprin...
1,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",파이썬 코드는 아래와 같고 from itertools import permutati...
2,0.26 x 0.8을 계산해 주세요.,파이썬 코드는 아래와 같고 a = 0.26\nb = 0.8\ny = a * b\np...
3,"현수네 집에서 미용실, 병원, 백화점까지의 거리는 각각 4/5km, 1/3km, 0...",파이썬 코드는 아래와 같고 a = 4 / 5\nb = 1 / 3\nc = 0.9\n...
4,어떤 수를 3으로 나누어야 하는 것을 잘못하여 9로 나누었더니 몫이 9가 되었습니다...,파이썬 코드는 아래와 같고 a = 3\nb = 9\nc = 9\ny = int(c ...
...,...,...
2015,어느 가전 마트에서는 115만 원인 노트북을 어제까지는 30%를 할인하여 판매하였습...,파이썬 코드는 아래와 같고 a = 1150000\nb = 30\nc = 15\nd ...
2016,10분 동안 6.53k미터를 가는 자동차와 20분 동안 15.12k미터를 가는 오토...,파이썬 코드는 아래와 같고 a = 6.53 / 10\nb = 15.12 / 20\n...
2017,4880보다 ○ 큰 수 는 9000입니다. ○안에 알맞은 수를 써 넣으시오.,파이썬 코드는 아래와 같고 a = 9000\nb = 4880\ny = a - b\n...
2018,어느 문구점에서 연필 한 자루는 400원이고 공책 한 권은 1000원입니다. 현승이...,파이썬 코드는 아래와 같고 a = 400\nb = 1000\nc = 5000\nd ...


In [195]:

extra_data_krx_sample_1_make_gpt_indexing = df_math_del8_clean.iloc[:,2:][['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset_1 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)



In [ ]:
extra_data_krx_sample_1_make_gpt__Dataset_1['prompt']

In [198]:
extra_data_krx_sample_1_make_gpt__Dataset_1.save_to_disk("data/final/krx-math1")

Saving the dataset (0/1 shards):   0%|          | 0/2020 [00:00<?, ? examples/s]

In [72]:
qa_df = pd.DataFrame(columns=["Q", "A"])

# Process each response and add rows directly to the DataFrame
for j in responses:
    try:
        # Split content and clean empty strings
        cleaned_data = [item for item in j['choices'][0]['message']['content'].split('\n') if item != '']
        
        # Check if data contains even pairs of Q & A
        if len(cleaned_data) % 2 == 0:
            for i in range(0, len(cleaned_data), 2):
                # Add each Q, A pair as a row in the DataFrame
                qa_df = pd.concat([qa_df, pd.DataFrame({"Q": [cleaned_data[i]], "A": [cleaned_data[i+1]]})], ignore_index=True)
    except Exception as e:
        print("Error processing response:", e)

NameError: name 'responses' is not defined

In [5]:
import csv
with open("krx_legal_extension_v1.csv", "r") as f:
    reader = csv.reader(f)
    loaded_data = next(reader)

qa_df_legal = pd.DataFrame(columns=["Q", "A"])

# Process each response and add rows directly to the DataFrame
for j in loaded_data:
    try:
        # Split content and clean empty strings
        cleaned_data = [item for item in j.split( 'content=')[1].split('\\n') if item != '']
        
        # Check if data contains even pairs of Q & A
        if len(cleaned_data) % 2 == 0:
            for i in range(0, len(cleaned_data), 2):
                # Add each Q, A pair as a row in the DataFrame
                qa_df_legal = pd.concat([qa_df_legal, pd.DataFrame({"Q": [cleaned_data[i]], "A": [cleaned_data[i+1]]})], ignore_index=True)
    except Exception as e:
        print("Error processing response:", e)

In [6]:
import pandas as pd
from datasets import Dataset,concatenate_datasets
from datasets import load_dataset



In [7]:
# qa_df.columns = ['question','response']
qa_df_legal.columns = ['question','response']

In [185]:

extra_data_krx_sample_1_make_gpt_indexing = qa_df_legal[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

In [189]:
extra_data_krx_sample_1_make_gpt__Dataset.save_to_disk("data/final/krx-legal")


Saving the dataset (0/1 shards):   0%|          | 0/1520 [00:00<?, ? examples/s]

In [16]:
import pandas as pd
from datasets import Dataset,concatenate_datasets
import pandas as pd
from datasets import Dataset,concatenate_datasets
from datasets import load_dataset

In [154]:
dataset_krx_sample = load_dataset("amphora/krx-sample-instructions", split = "train") # 일반적인 경제 상식 25951개

In [156]:
dataset_krx_sample.save_to_disk("data/final/krx-sample_normal_finance")

Saving the dataset (0/1 shards):   0%|          | 0/25951 [00:00<?, ? examples/s]

In [157]:
extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant.csv') #금융 공학 내용 1000개
extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset_1 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)
extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

C:\Users\son\AppData\Local\Temp\ipykernel_10340\209054034.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index


In [158]:
extra_data_krx_sample_1_make_gpt__Dataset

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 1000
})

In [161]:
extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant.csv') #금융 공학 내용 1000개
extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset_1 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)
print(len(extra_data_krx_sample_1_make_gpt))
extra_data_krx_sample_1_make_gpt__Dataset_1 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant_final.csv') #금융 공학 내용 4000개
extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset_2 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

combined_dataset = concatenate_datasets([extra_data_krx_sample_1_make_gpt__Dataset_1,extra_data_krx_sample_1_make_gpt__Dataset_2])

extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant_final_extention_v1.csv') #금융 공학 내용 16200개
extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
extra_data_krx_sample_1_make_gpt__Dataset_3 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

combined_dataset_1 = concatenate_datasets([combined_dataset,extra_data_krx_sample_1_make_gpt__Dataset_3])

C:\Users\son\AppData\Local\Temp\ipykernel_10340\1296983350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
C:\Users\son\AppData\Local\Temp\ipykernel_10340\1296983350.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index


1000


C:\Users\son\AppData\Local\Temp\ipykernel_10340\1296983350.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index


In [162]:
combined_dataset_1

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 21952
})

In [163]:
combined_dataset_1.save_to_disk("data/final/krx-sample_rewrite_financial_engineering")

Saving the dataset (0/1 shards):   0%|          | 0/21952 [00:00<?, ? examples/s]

In [181]:
# # # dataset_krx_sample = load_dataset("amphora/krx-sample-instructions", split = "train") # 일반적인 경제 상식 25951개
# # # 추가 데이터 로드 후 결합
# # extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant.csv') #금융 공학 내용 1000개
# # extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
# # extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
# # extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
# # extra_data_krx_sample_1_make_gpt__Dataset_1 = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)
# # print(len(extra_data_krx_sample_1_make_gpt))
# # extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

# extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant_final.csv') #금융 공학 내용 4000개
# extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
# extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
# extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
# extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

# combined_dataset = concatenate_datasets([combined_dataset,extra_data_krx_sample_1_make_gpt__Dataset])

# extra_data_krx_sample_1_make_gpt = pd.read_csv('c:/Users/son/Desktop/KRX_compatition/rewrite-se-quant_final_extention_v1.csv') #금융 공학 내용 16200개
# extra_data_krx_sample_1_make_gpt_indexing = extra_data_krx_sample_1_make_gpt[['question','response']]
# extra_data_krx_sample_1_make_gpt_indexing['__index_level_0__'] = extra_data_krx_sample_1_make_gpt_indexing.index
# extra_data_krx_sample_1_make_gpt_indexing.columns = ['prompt','response','__index_level_0__']
# extra_data_krx_sample_1_make_gpt__Dataset = Dataset.from_pandas(extra_data_krx_sample_1_make_gpt_indexing)

# combined_dataset = concatenate_datasets([combined_dataset,extra_data_krx_sample_1_make_gpt__Dataset])
qa_df = pd.read_csv('kor_company_data_69827.csv')
extra_korcompany_gpt4o_indexing = qa_df.iloc[:,1:]
extra_korcompany_gpt4o_indexing.columns = ['question','response'] # 한국 기업 재무정보 68989개
extra_korcompany_gpt4o_indexing['__index_level_0__'] = extra_korcompany_gpt4o_indexing.index
extra_korcompany_gpt4o_indexing.columns = ['prompt','response','__index_level_0__']
extra_extra_korcompany_gpt4o_Dataset = Dataset.from_pandas(extra_korcompany_gpt4o_indexing)

# combined_dataset = combined_dataset.map(formatting_prompts_func, batched = True)

In [9]:
dataset_krx_sample = load_dataset("amphora/krx-sample-instructions", split = "train") # 일반적인 경제 상식 25951개
extra_krxlegal_gpt4o_indexing = qa_df_legal[['question','response']] # 법 내용 1000개
extra_krxlegal_gpt4o_indexing['__index_level_0__'] = extra_krxlegal_gpt4o_indexing.index
extra_krxlegal_gpt4o_indexing.columns = ['prompt','response','__index_level_0__']
extra_extra_krxlegal_gpt4o_Dataset = Dataset.from_pandas(extra_krxlegal_gpt4o_indexing)
combined_dataset = concatenate_datasets([dataset_krx_sample,extra_extra_krxlegal_gpt4o_Dataset])

In [10]:
combined_dataset

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 27471
})

In [1]:
from datasets import load_from_disk

# 로컬 경로에서 데이터셋 불러오기
combined_dataset = load_from_disk("c:/Users/son/Desktop/KRX_compatition/data/final_dataset/LORA1/krx-legal")

FileNotFoundError: Directory c:/Users/son/Desktop/KRX_compatition/data/final_dataset/LORA1/krx-legal not found

In [ ]:
combined_dataset['prompt']

In [342]:
legal_1 = pd.read_csv("newFE.csv")

In [343]:
legal_1

,Unnamed: 0,Question,Answer
0,0,"미국형 콜 옵션의 가격을 이항 가격 결정 모델을 통해 산출할 때, 배당금을 지급하지...","배당금을 지급하지 않는 경우, 조기 행사 가치와 만기 시점의 미래 가치를 비교해야 ..."
1,1,유럽형 콜 옵션의 가격은 어떻게 계산되나요?,유럽형 콜 옵션의 가격은 만기 시점에만 행사할 수 있는 옵션이므로 다음과 같은 식으...
2,2,미국형 콜 옵션의 가격을 결정할 때 어떤 식을 사용하나요?,미국형 콜 옵션의 가격은 다음 방정식으로 표현됩니다: \(V_n(\omega) = ...
3,3,만료되지 않은 심각한 인 더 머니(deep in-the-money) 상태의 미국형 ...,"이 경우 옵션의 현재 가치인 \(S(\omega) - K\)는 매우 높지만, 조기 ..."
4,4,배당금을 지급하지 않는 주식에서 미국형 콜 옵션과 유럽형 콜 옵션의 가격이 동일하다...,"배당금이 없는 경우 조기 행사로 인한 기회비용이 발생하지 않으며, 따라서 시간 가치..."
...,...,...,...
123385,123385,포워드 가격이란 무엇인가요?,포워드 가격은 현재 시점에서 미래의 특정 시점에 한정된 금액을 특정 환율로 교환하기...
123386,123386,포워드 가격은 어떻게 계산되나요?,"포워드 가격은 현재 스팟 환율과 두 나라의 이자율을 바탕으로 계산되며, 다음의 비율..."
123387,123387,산술 평균 환율을 사용한 포워드 가격 계산은 어떻게 이루어지나요?,"산술 평균 환율을 사용하는 경우, 일정 기간 동안의 스팟 환율을 먼저 평균하여 구한..."
123388,123388,기업이나 투자자가 FX 포지션을 관리할 때 평균 환율을 고려하는 이유는 무엇인가요?,평균 환율을 고려하는 것은 환율 변동으로 인한 리스크를 줄이는 한 방법입니다. 이를...


In [326]:
legal_1_list = legal_1.columns.to_list()

In [327]:
# 정규표현식 패턴 정의
qa_pattern = re.compile(r"Q\.\s*(.*?)\s*A\.\s*(.*?)(?=Q\.|$)", re.DOTALL)


In [337]:
# Q&A 데이터를 저장할 리스트 초기화
all_questions = []
all_answers = []

# 데이터 리스트를 순회하며 Q&A 추출
for text in legal_1_list:
    # 정규표현식을 사용하여 Q&A 쌍 찾기
    qa_pairs = qa_pattern.findall(text)
    for q, a in qa_pairs:
        # 질문과 답변의 공백 및 줄바꿈 문자 정리
        question = q.strip().replace('\n', ' ').replace('\\n', ' ')
        answer = a.strip().replace('\n', ' ').replace('\\n', ' ')
        all_questions.append(question)
        all_answers.append(answer)

# Q&A 데이터를 DataFrame으로 변환
qa_df = pd.DataFrame({
    "Question": all_questions,
    "Answer": all_answers
})

# 결과 확인
print(qa_df)


                                               Question  \
0     거래소거래확인시세텀 운영 등에 대한 내용이 담긴...   
1     거래소거래확인시세텀 운영 등에 사용되는 “이용자...   
2     거래소거래확인시세텀 운영 등에 “거래확인서”란 ...   
3     거래소거래확인시세텀 운영 등에 기준의 변경 사항...   
4     거래소거래확인시세텀 운영 등에 "당사자이용자"의...   
...                                                 ...   
1632  참가자가 매매거래에 관하여 거래증명을 요청할 경우, 거래소는 어떻게 처리해야 하나요...   
1633                   KRX석유시장에서 발생한 이자는 누구에게 귀속되나요?      
1634  KRX석유시장 활성화를 위해 거래소는 어떤 방식으로 참가자에게 대가를 지급할 수 있...   
1635        KRX석유시장 참가자는 별도의 약정을 통해 어떤 사항을 정할 수 있나요?      
1636              KRX석유시장 규정의 시행에 관한 사항은 무엇으로 정해지나요?      

                                                 Answer  
0     제1조의 목적은 한국거래소「장외파생상품 청산업무규정 시행세칙」에 따라 거래소거래확인...  
1        “이용자”란 한국거래소와 거래소거래확인시스템 이용계약을 체결한 자를 의미합니다.    
2     “거래확인서”란 기본계약의 적용을 받는 개별거래에 대한 거래조건 등 합의내용을 서면...  
3     기준의 변경 사항은 거래소 홈페이지 및 시스템 화면에 게시하거나 기타의 방법으로 이...  
4

In [338]:
qa_df

,Question,Answer
0,거래소거래확인시세텀 운영 등에 대한 내용이 담긴...,제1조의 목적은 한국거래소「장외파생상품 청산업무규정 시행세칙」에 따라 거래소거래확인...
1,거래소거래확인시세텀 운영 등에 사용되는 “이용자...,“이용자”란 한국거래소와 거래소거래확인시스템 이용계약을 체결한 자를 의미합니다.
2,거래소거래확인시세텀 운영 등에 “거래확인서”란 ...,“거래확인서”란 기본계약의 적용을 받는 개별거래에 대한 거래조건 등 합의내용을 서면...
3,거래소거래확인시세텀 운영 등에 기준의 변경 사항...,기준의 변경 사항은 거래소 홈페이지 및 시스템 화면에 게시하거나 기타의 방법으로 이...
4,"거래소거래확인시세텀 운영 등에 ""당사자이용자""의...",“당사자이용자”란 이용자 중 기본계약에 의거하여 체결된 거래의 당사자를 의미합니다....
...,...,...
1632,"참가자가 매매거래에 관하여 거래증명을 요청할 경우, 거래소는 어떻게 처리해야 하나요...",거래소는 참가자의 요청에 따라 지체 없이 서면 또는 전자문서로 거래증명을 발급해야 ...
1633,KRX석유시장에서 발생한 이자는 누구에게 귀속되나요?,KRX석유시장에서 발생한 이자는 거래소에 귀속됩니다.
1634,KRX석유시장 활성화를 위해 거래소는 어떤 방식으로 참가자에게 대가를 지급할 수 있...,거래소는 KRX석유시장 활성화에 기여한 참가자에게 석유시장 수수료의 범위 내에서 대...
1635,KRX석유시장 참가자는 별도의 약정을 통해 어떤 사항을 정할 수 있나요?,KRX석유시장 참가자는 계약서 및 규정에 위반되지 않는 범위 내에서 석유제품의 매매...


In [344]:
extra_krxlegal_gpt4o_indexing = legal_1[['Question','Answer']] # 법 내용 1000개
extra_krxlegal_gpt4o_indexing['__index_level_0__'] = extra_krxlegal_gpt4o_indexing.index
extra_krxlegal_gpt4o_indexing.columns = ['prompt','response','__index_level_0__']
extra_extra_krxlegal_gpt4o_Dataset = Dataset.from_pandas(extra_krxlegal_gpt4o_indexing)


In [345]:
extra_extra_krxlegal_gpt4o_Dataset

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 123390
})

In [346]:
extra_extra_krxlegal_gpt4o_Dataset.save_to_disk('krx-FE')

Saving the dataset (0/1 shards):   0%|          | 0/123390 [00:00<?, ? examples/s]

In [335]:
def clean_text(text):
    # 불필요한 공백 제거
    text = text.strip()
    # 연속된 공백을 단일 공백으로 변환
    text = re.sub(r'\s+', ' ', text)
    return text

# 클렌징 적용
qa_df['Question'] = qa_df['Question'].apply(clean_text)
qa_df['Answer'] = qa_df['Answer'].apply(clean_text)

# 클렌징된 데이터 확인


In [336]:
qa_df['Answer'][1636]

"KRX석유시장 규정의 시행에 관한 필요한 사항은 세칙으로 정해집니다.', role='assistant', tool_calls=None, function_call=None))], created=1731915212, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_0ba0d124f1', usage=Usage(completion_tokens=473, prompt_tokens=1318, total_tokens=1791, completion_tokens_details=CompletionTokensDetailsWrapper(audio_tokens=0, reasoning_tokens=0, text_tokens=None, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier=None)"

In [320]:
combined_dataset['response'][706]

'A. 정규시장은 9시부터 15시 30분까지이며, 단, 시장관리상 필요에 따라 거래소가 정하는 경우에는 임시로 변경될 수 있다.'

In [166]:
combined_dataset['prompt'][-1]

'Q7. KRX석유시장과 관련된 규정의 시행에 관하여 필요한 사항은 무엇으로 정하는가?'

In [167]:
qa_df = pd.DataFrame([combined_dataset['prompt'][26951:],combined_dataset['response'][26951:]])

In [171]:
qa_df.T.columns = ['question','response']

In [172]:
qa_df

,0,1,2,3,4,5,6,7,8,9,...,69817,69818,69819,69820,69821,69822,69823,69824,69825,69826
0,Q. 노바텍의 2023년 연결재무상태표에서 자산총계가 증가한 원인은 무엇인가요?,"Q. 2023년 노바텍의 부채총계는 어떻게 변화하였으며, 그 원인은 무엇인가요?","Q. 2023년 노바텍의 영업이익률은 얼마였으며, 이전 연도와 비교했을 때 어떤 변...","Q. 노바텍의 당기순이익은 2023년에 어떻게 변화했으며, 그 증가의 주된 원인은 ...","Q. 노바텍의 2023년 유동비율은 얼마이며, 이는 전년도와 비교하여 어떻게 변화하...",Q. 노바텍의 신규사업으로 시작한 친환경 Recycle Plate 제작의 주요 목표...,Q. 환율변동이 노바텍의 재무에 미치는 영향은 어떤가요?,Q. 노바텍의 2023년도 ROA와 ROE는 각각 얼마인가요?,Q. 노바텍은 자산 손상징후를 검토할 때 어떤 절차를 따라야 하나요?,"Q. 노바텍이 보유하고 있는 유동자금의 규모는 얼마이며, 이전 연도와 비교했을 때 ...",...,Q. KRX석유시장 운영규정에 따른 매매거래 상황 파악의 목...,Q. KRX석유시장 운영규정에서 일반상품시장위원회의 역할은 ...,Q. KRX석유시장 운영규정에 따라 일반상품시장위원회의 위원...,'Q1. KRX석유시장 참가자는 석유제품담당자를 몇 명 이상 지정하여 거래소에 등록...,Q2. KRX석유시장 주유소 협의회는 어떤 목적으로 설치되는가?,Q3. 참가자는 매매거래에 관한 거래증명을 어떻게 요청할 수 있는가?,Q4. 거래소 명의의 결제계좌에서 발생한 이자는 누구에게 귀속되는가?,Q5. KRX석유시장에 활성화를 기여한 참가자에게 거래소가 지급할 수 있는 대가는 ...,Q6. 참가자는 계약서 및 규정에 위반되지 않는 범위 내에서 어떤 사항에 대해 별도...,Q7. KRX석유시장과 관련된 규정의 시행에 관하여 필요한 사항은 무엇으로 정하는가?
1,"A. 자산총계는 유동자산과 비유동자산 모두에서 증가하였으며, 특히 기타유동금융자산,...","A. 2023년 부채총계는 105억 원으로, 2022년 대비 22억 원이 감소하였으...","A. 2023년 영업이익률은 27.7%로, 전년의 24%에 비해 증가하였습니다.","A. 2023년 당기순이익은 296억 원으로, 전년 대비 153억 원이 증가하였으며...","A. 2023년 유동비율은 1685.64%로, 전년의 1167.29%와 비교하여 상...","A. 신규사업의 주요 목표는 개발 및 영업 인력을 충원하여 사업을 확장하고, 차별화...","A. 환율변동은 영업 거래, 자산 및 부채에 영향을 미칠 수 있으며, 이를 최소화하...","A. 2023년도 ROA는 17.5%, ROE는 18.7%입니다.","A. 자산의 손상징후를 검토하고, 손상징후가 있을 경우 자산의 회수가능액과 장부가액...","A. 2023년 말 유동자금은 1,002억 원으로, 전년의 856억 원 대비 146...",...,A. 매매거래 조치의 적시성·적정성 등을 확보하기 위해 거래소는 상시적으로 매매거래...,"A. 일반상품시장위원회는 시장 제도의 개선, 석유제품 종류의 변경 등 시장운영과 관...",A. 일반상품시장위원회의 위원 구성 및 회의 개최 등 운영에 관한 필요한 사항은 거...,A1. 참가자는 1인 이상의 석유제품담당자를 지정하여 거래소에 등록해야 한다.,A2. KRX석유시장 주유소 협의회는 KRX석유시장의 발전을 위한 공동 노력 및 상...,"A3. 참가자는 거래소에 매매거래에 관한 거래증명을 요청할 수 있으며, 거래소는 지...",A4. 거래소 명의의 결제계좌에서 발생한 이자는 거래소에 귀속된다.,A5. 거래소는 KRX석유시장 수수료의 범위 내에서 대가를 지급할 수 있다.,A6. 참가자는 석유제품의 매매거래와 관련한 사항에 대해 다른 참가자와의 별도 약정...,A7. KRX석유시장과 관련된 규정의 시행에 관하여 필요한 사항은 세칙으로 정한다....


In [70]:
qa_df.T.to_csv('kor_company_data_69827.csv')

In [173]:
qa_df = pd.read_csv('kor_company_data_69827.csv')

In [174]:
qa_df

,Unnamed: 0,0,1
0,0,Q. 노바텍의 2023년 연결재무상태표에서 자산총계가 증가한 원인은 무엇인가요?,"A. 자산총계는 유동자산과 비유동자산 모두에서 증가하였으며, 특히 기타유동금융자산,..."
1,1,"Q. 2023년 노바텍의 부채총계는 어떻게 변화하였으며, 그 원인은 무엇인가요?","A. 2023년 부채총계는 105억 원으로, 2022년 대비 22억 원이 감소하였으..."
2,2,"Q. 2023년 노바텍의 영업이익률은 얼마였으며, 이전 연도와 비교했을 때 어떤 변...","A. 2023년 영업이익률은 27.7%로, 전년의 24%에 비해 증가하였습니다."
3,3,"Q. 노바텍의 당기순이익은 2023년에 어떻게 변화했으며, 그 증가의 주된 원인은 ...","A. 2023년 당기순이익은 296억 원으로, 전년 대비 153억 원이 증가하였으며..."
4,4,"Q. 노바텍의 2023년 유동비율은 얼마이며, 이는 전년도와 비교하여 어떻게 변화하...","A. 2023년 유동비율은 1685.64%로, 전년의 1167.29%와 비교하여 상..."
...,...,...,...
69822,69822,Q3. 참가자는 매매거래에 관한 거래증명을 어떻게 요청할 수 있는가?,"A3. 참가자는 거래소에 매매거래에 관한 거래증명을 요청할 수 있으며, 거래소는 지..."
69823,69823,Q4. 거래소 명의의 결제계좌에서 발생한 이자는 누구에게 귀속되는가?,A4. 거래소 명의의 결제계좌에서 발생한 이자는 거래소에 귀속된다.
69824,69824,Q5. KRX석유시장에 활성화를 기여한 참가자에게 거래소가 지급할 수 있는 대가는 ...,A5. 거래소는 KRX석유시장 수수료의 범위 내에서 대가를 지급할 수 있다.
69825,69825,Q6. 참가자는 계약서 및 규정에 위반되지 않는 범위 내에서 어떤 사항에 대해 별도...,A6. 참가자는 석유제품의 매매거래와 관련한 사항에 대해 다른 참가자와의 별도 약정...


In [151]:
combined_dataset = concatenate_datasets([combined_dataset,combined_dataset_math])

In [3]:
extra_data_krx_sample_1_make_gpt__Dataset

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 4000
})

In [152]:
combined_dataset

Dataset({
    features: ['prompt', 'response', '__index_level_0__'],
    num_rows: 120782
})

In [153]:
combined_dataset.save_to_disk("data/final_Asample_Akorcompany_Alegal_cfa_kim")

Saving the dataset (0/1 shards):   0%|          | 0/120782 [00:00<?, ? examples/s]

In [ ]:
combined_dataset['prompt']

In [235]:

number1 = pd.read_csv('data/finance_kor_exam/서기쌤 파생최종 데이터셋(1).csv',encoding='euc-kr')
number2 = pd.read_csv('data/finance_kor_exam/서기쌤 파생최종 데이터셋(2) (최종본).csv',encoding='euc-kr')
number3 = pd.read_csv('data/finance_kor_exam/증투사 2과목 데이터셋(최종).csv',encoding='euc-kr')
number4 = pd.read_csv('data/finance_kor_exam/증투사 100% 데이터셋 (1).csv',encoding='euc-kr')
number5 = pd.read_csv('data/finance_kor_exam/펀드투자 데이터셋.csv',encoding='euc-kr')



In [233]:
number5 = pd.read_csv('data/finance_kor_exam/펀드투자 데이터셋.csv', encoding='euc-kr', on_bad_lines='skip')


In [240]:
import re
def adjust_spaces(text):
    # 1개의 공백 제거
    text = re.sub(r'(?<!\s)\s(?!\s)', '', text)  # 공백 1개만 제거
    # 2개 이상의 공백을 1개로 줄이기
    text = re.sub(r'\s{2,}', ' ', text)
    return text

# 데이터프레임의 특정 열에 함수 적용
number1['Answer'] =  number1['Answer'].apply(adjust_spaces)

In [241]:
number1['Answer'][0]

'D. 선도거래는 청산소에서 계약이행을 보증한다. 해설. 선도거래는 장외거래이다. '

In [243]:
df_combined = pd.concat([number1, number2, number3, number4, number5], axis=0, ignore_index=True)

In [246]:
df_combined.columns = ['question','response']

In [247]:
df_combined.columns = ['question','response']
extra_krxlegal_gpt4o_indexing = df_combined[['question','response']] # 법 내용 1000개
extra_krxlegal_gpt4o_indexing['__index_level_0__'] = extra_krxlegal_gpt4o_indexing.index
extra_krxlegal_gpt4o_indexing.columns = ['prompt','response','__index_level_0__']
extra_extra_korexam_gpt4o_Dataset = Dataset.from_pandas(extra_krxlegal_gpt4o_indexing)


In [250]:
extra_extra_korexam_gpt4o_Dataset.save_to_disk("data/final_dataset/korexam")

Saving the dataset (0/1 shards):   0%|          | 0/595 [00:00<?, ? examples/s]